In [4]:
from qiskit.opflow import PrimitiveOp, PauliTrotterEvolution
from qiskit.quantum_info import SparsePauliOp
from qiskit import transpile, QuantumCircuit

def ops_from_file(filename, num_lines=False):
    """Read Hamiltonian from file and return list of qiskit.opflow.PrimitiveOp
    File format: 'coefficient Pauli-string' per line, separated by space
    Optional parameter num_lines: only read first # operators
    """
    try:
        file = open(filename) # read file
    except:
        print("An exception occurred trying to read the file: " + filename) 
    lines = file.read().split('\n')
    i = 1
    ret = []
    for l in lines:
        if num_lines and i > num_lines: # reached max. number lines
            return ret
        if len(l): # exclude empty lines
            [c,o] = l.split() # split in coefficient c and operator string o
            ret.append(PrimitiveOp(SparsePauliOp(o, float(c)))) 
            i += 1
    return ret

def ops_to_circ(ops):
    """Converts list of PrimitiveOp to list of quantum circuits"""
    circs = []
    for o in ops:
        unitary = PauliTrotterEvolution().convert(o.exp_i())
        circs.append(unitary.to_circuit().decompose())
    return circs

def circ_depths(circs):
    """Returns list of depths for transpiled circuits in list circs"""
    depths = []
    for qc in circs:
        qct = transpile(qc, basis_gates=['u','cx'])
        depths.append(qct.depth())
    return depths

qcs = ops_to_circ(ops_from_file("hamiltonian_ordered.txt", 191))
dep = circ_depths(qcs)

In [3]:
print(dep)
sum(dep)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 13, 13, 13, 13, 13, 13, 13, 13, 21, 21, 21, 21, 9, 9, 9, 9, 11, 11, 11, 11, 7, 7, 7, 7, 11, 11, 11, 11, 7, 7, 7, 7, 13, 13, 13, 13, 9, 9, 9, 9, 11, 11, 11, 11, 15, 15, 15, 15, 15, 15, 15, 15, 19, 19, 19, 19, 19, 19, 19, 19, 17, 17, 17, 17, 17, 17, 17, 17, 13, 13, 13, 13, 11, 11, 11, 11, 5, 5, 5, 5, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 9, 9, 9, 9, 13, 13, 13, 13, 15, 15, 15, 15, 11, 11, 11, 11, 15, 15, 15, 15, 11, 11, 11, 11, 9, 9, 9, 9, 11, 11, 11, 11, 9, 9, 9, 9, 7, 7, 7]


1810

In [13]:
# Transpile only at the end
qc = QuantumCircuit(10)
for i in range(len(qcs)-1,-1,-1): # unitary U1 * U2 * ... is constructed by appending circuits in reverse order
    qc.append(qcs[i],range(10))
qct = transpile(qc, basis_gates=['u','cx'])
print(qct.depth())

1430


In [14]:
# Transpile in every step
qc = QuantumCircuit(10)
for i in range(len(qcs)-1,-1,-1): # unitary U1 * U2 * ... is constructed by appending circuits in reverse order
    qc.append(transpile(qcs[i], basis_gates=['u','cx']),range(10))
qct = transpile(qc, basis_gates=['u','cx'])
print(qct.depth())

1430


In [52]:
# no transpile
qc = QuantumCircuit(10)
for i in range(len(qcs)-1,-1,-1): # unitary U1 * U2 * ... is constructed by appending circuits in reverse order
    qc.append(qcs[i],range(10))

In [15]:
import numpy as np
import scipy.sparse as sparse
from numpy import cos, sin, exp, pi
from scipy.sparse.linalg import expm, eigs
from qiskit import transpile, QuantumCircuit
from qiskit.opflow import PrimitiveOp, PauliTrotterEvolution
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit import Parameter

param_t = Parameter('t')

def char_to_pauli(char):
    """Returns the Pauli matrix for a given character in I, X, Y, Z"""
    char = char.upper() # convert lower to upper case
    if char == 'I':
        return np.matrix([[1,0],[0,1]])
    if char == 'X':
        return np.matrix([[0,1],[1,0]])
    if char == 'Y':
        return np.matrix([[0,-1j],[1j,0]])
    if char == 'Z':
        return np.matrix([[1,0],[0,-1]])
    raise Exception("Character '" + char + "' does not correspond to Pauli matrix")

def string_to_operator(string):
    """Calculates the operator (Kronecker product) from a Pauli string"""
    op = sparse.csc_matrix(1, dtype=np.complex128)
    for c in string:
        op = sparse.kron(op, char_to_pauli(c))
    return op

class PauliHamiltonian:
    """Class for Hamiltonian built from Pauli strings
    Attributes:
        operators: list of Pauli strings
        coefficients: dictionary {'Pauli String': coefficient}
        qubits: number of qbits 
    """
    operators = []
    coefficients = {}
    qubits = 0
    
    __matrix = None
    __unitary = None
    __gates = {}
    #__gate_unitaries = {}
    
    def matrix(self):
        """Hamiltonian in matrix form"""
        if self.__matrix is None: # matrix form has not been calculated yet
            n = self.dim()
            self.__matrix = sparse.csc_matrix((n, n), dtype=np.complex128)
            for o in self.operators:
                self.__matrix += self.coefficients[o] * string_to_operator(o)
        return self.__matrix
    
    def unitary(self):
        """exp(-iH) in matrix form"""
        if self.__unitary is None: # unitary has not been calculated yet
            self.__unitary = expm(-1j * self.matrix())
        return self.__unitary
    
    def pauli_gate(self, pauli):
        """Gate corresponding to the single trotter step with Pauli string pauli"""
        if pauli not in self.__gates: # gates have not been calculated yet
            ham = PrimitiveOp(SparsePauliOp(pauli, self.coefficients[pauli])) # Hamiltonian
            ut = PauliTrotterEvolution().convert((param_t*ham).exp_i()) # single Trotter step
            self.__gates[pauli] = transpile(ut.to_circuit(), basis_gates=['u','cx']) # turn into circuit with only allowed gates
        return self.__gates[pauli]
    
    def pauli_gate_unitary(self, pauli, t=1):
        """Unitary matrix corresponding to the single trotter step with Pauli string pauli and t=1"""
        if pauli == 'I' * self.qubits: # identity
            return exp(-1j*self.coefficients[pauli]*t)*sparse.eye(self.dim(), format='csc')
        qc = self.pauli_gate(pauli)
        qc = qc.bind_parameters({param_t: t})
        return sparse.csc_matrix(Operator(qc).data, dtype=np.complex128)
    
    def coefficient(self, operator):
        """Coefficient that goes with the Pauli String operator"""
        return self.coefficients[operator]
    
    def error(self, unitary, tol=0):
        """Largest abs eigenvalue of difference between this Hamiltonian and
           unitary input.
           unitary must be of.sparse csc_matrix type.
           Parameter tol sets precision of eigenvalues (0=machine precision)"""
        diff = self.unitary() - unitary
        ev = eigs(diff, 1, which='LM', return_eigenvectors=False)
        return np.abs(ev[0])   
    
    def circuit_error(self, circuit):
        """Error made by circuit"""
        circuit.global_phase = -self.coefficients['I'*self.qubits]
        return self.error(sparse.csc_matrix(Operator(circuit).data, dtype=np.complex128))
    
    def dim(self):
        """Matrix dimension 2^n with n qubits"""
        return 2**self.qubits
    
    def trotterize(self, steps):
        """Unitary matrix corresponding to steps Trotter steps"""
        utt = sparse.eye(self.dim(), format='csc')
        t = 1/steps
        for o in self.operators:
            utt = utt.dot(self.pauli_gate_unitary(o, t))
        ut = sparse.eye(self.dim(), format='csc')
        for i in range(steps):
            ut = ut.dot(utt)
        return ut
    
    def trotter_error(self, steps, tol=0):
        return self.error(self.trotterize(steps), tol)
        
            
def hamiltonian_from_file(filename):
    """Read Hamiltonian from file and return PauliHamiltonian object.
    File format: 'coefficient Pauli-string' per line, separated by space"""
    try:
        file = open(filename) # read file
    except:
        print("An exception occurred trying to read the file: " + filename) 
    lines = file.read().split('\n')
    h = PauliHamiltonian() # create object to return
    for l in lines:
        if len(l): # exclude empty lines
            [c,o] = l.split() # split in coefficient c and operator string o
            if h.qubits:
                if len(o) != h.qubits: # Make sure all operators have the same dimension
                    raise Exception("Operator string '" + o + "' does not match dimension (" + h.qubits + ")")
            else:
                h.qubits = len(o) # set no. qubits to length of operator string
            h.operators.append(o) # add operator to list of operators of object h
            h.coefficients[o] = float(c) # add to the dictionary of coefficients for h
    return h

h = hamiltonian_from_file("hamiltonian_ordered.txt")

In [17]:
# Check unitary
from qiskit import Aer, execute

qct.global_phase = -1.0709274663656798

backend = Aer.get_backend('unitary_simulator')
job = execute(qct, backend)
result = job.result()
u = sparse.csc_matrix(result.get_unitary(qct), dtype=np.complex128)
h.error(u)

0.8993411108510606

In [ ]:
# Check unitary for no transpile
from qiskit import Aer, execute

backend = Aer.get_backend('unitary_simulator')
job = execute(qc, backend)
result = job.result()
u = sparse.csc_matrix(result.get_unitary(qc), dtype=np.complex128)
h.error(u)

In [29]:
ops = ops_from_file("hamiltonian_ordered.txt", 191)
qcs = ops_to_circ(ops)

In [31]:
ops[1]

PauliSumOp(SparsePauliOp(['IIIIIZIIII'],
              coeffs=[-0.5772921+0.j]), coeff=1.0)

In [32]:
print(qcs[1])

                    
q_0: ───────────────
                    
q_1: ───────────────
                    
q_2: ───────────────
                    
q_3: ───────────────
     ┌─────────────┐
q_4: ┤ Rz(-1.1546) ├
     └─────────────┘
q_5: ───────────────
                    
q_6: ───────────────
                    
q_7: ───────────────
                    
q_8: ───────────────
                    
q_9: ───────────────
                    


In [44]:
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import expm, eigs
from qiskit.opflow import PrimitiveOp, PauliTrotterEvolution
from qiskit.quantum_info import SparsePauliOp

def char_to_pauli(char):
    """Returns the Pauli matrix for a given character in I, X, Y, Z"""
    char = char.upper() # convert lower to upper case
    if char == 'I':
        return np.matrix([[1,0],[0,1]])
    if char == 'X':
        return np.matrix([[0,1],[1,0]])
    if char == 'Y':
        return np.matrix([[0,-1j],[1j,0]])
    if char == 'Z':
        return np.matrix([[1,0],[0,-1]])
    raise Exception("Character '" + char + "' does not correspond to Pauli matrix")

def string_to_operator(string):
    """Calculates the operator (Kronecker product) from a Pauli string"""
    op = sparse.csc_matrix(1, dtype=np.complex128)
    for c in string:
        op = sparse.kron(op, char_to_pauli(c))
    return op

def pauli_error(string, coeff):
    # first calculate the unitary of the quantum circuit
    op = PrimitiveOp(SparsePauliOp(string, float(coeff)))
    unitary = PauliTrotterEvolution().convert(op.exp_i())
    qc = unitary.to_circuit().decompose()
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    uc = sparse.csc_matrix(result.get_unitary(qc), dtype=np.complex128)
    # then unitary matrix
    um = expm(-1j * coeff * string_to_operator(string))
    if np.array_equal(uc.toarray(), um.toarray()):
        return 0 # matrices exactly equal
    # now the difference
    diff = uc - um
    # and error
    ev = eigs(diff, 1, which='LM', return_eigenvectors=False)
    return np.abs(ev[0])

In [47]:
pauli_error('IIIIIIIIII', 0.1)

1.6711069443220838e-16